In [80]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.python.keras.models import Sequential, Model, load_model
from tensorflow.python.keras.layers import Dense, Input, Dropout, Conv2D, Flatten, LSTM, Conv1D
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# 1. data
dataset = pd.read_csv('./train.csv')
submission_sample = pd.read_csv('./sample_submission.csv')

dataset.drop(['종목명'], axis=1, inplace=True)
dataset['일자'] = pd.to_datetime(dataset['일자'], format='%Y%m%d')
dataset = dataset[dataset['일자'].dt.year == 2023]
#dataset = dataset[(dataset['일자'].dt.year == 2023) | (dataset['일자'].dt.year == 2022)]
dataset = dataset.sort_values(by=['종목코드', '일자'], ascending=[True, True])
stock_codes = dataset['종목코드'].unique()
train = dataset.copy()

In [81]:
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import Sequential
from tensorflow.keras.activations import sigmoid

In [82]:


dropped_samples = pd.DataFrame()

print(train['종목코드'].nunique())

stocks_to_drop = train[train['시가'] == 0]['종목코드'].unique()
dropped_samples = pd.concat([dropped_samples, train[train['종목코드'].isin(stocks_to_drop)]])

train = train[~train['종목코드'].isin(stocks_to_drop)]

print(train['종목코드'].nunique())

code_counts = train['종목코드'].value_counts()

max_count = code_counts.max()

stocks_to_drop = code_counts[code_counts < max_count].index
dropped_samples = pd.concat([dropped_samples, train[train['종목코드'].isin(stocks_to_drop)]])

train = train[~train['종목코드'].isin(stocks_to_drop)]

print(train['종목코드'].nunique())

train['종가변동률'] = train.groupby('종목코드')['종가'].pct_change()

stocks_to_drop = train[train['종가변동률'] <= -0.5]['종목코드'].unique()
dropped_samples = pd.concat([dropped_samples, train[train['종목코드'].isin(stocks_to_drop)]])

train = train[~train['종목코드'].isin(stocks_to_drop)]
train.drop(['종가변동률'],axis=1,inplace=True)

print(train['종목코드'].nunique())
train['일자'] = pd.to_datetime(train['일자'])  # 날짜 형식이 맞는지 확인
train = train.sort_values(by=['종목코드', '일자'])  # 주식 종목과 날짜 순으로 정렬


# shift를 이용해 n일 전의 거래량을 가져옵니다.
for i in range(1, 16):
    train[f'거래량_lag_{i}'] = train.groupby('종목코드')['거래량'].transform(lambda x: x.shift(i)).fillna(0)

train['3일_거래량_증가_여부'] = train.apply(lambda row: 1 if all(row[f'거래량_lag_{i}'] < row[f'거래량_lag_{i+1}'] for i in range(1, 3)) else 0, axis=1)


train['7일_거래량_증가_여부'] = train.apply(lambda row: 1 if all(row[f'거래량_lag_{i}'] < row[f'거래량_lag_{i+1}'] for i in range(1, 7)) else 0, axis=1)

# 15일전부터 거래량이 꾸준히 상승하는지 판단하는 피처 생성
#train['15일_거래량_증가_여부'] = train.apply(lambda row: 1 if all(row[f'거래량_lag_{i}'] < row[f'거래량_lag_{i+1}'] for i in range(1, 15)) else 0, axis=1)





train['3일_지수이동평균'] = train.groupby('종목코드')['종가'].transform(lambda x: x.ewm(span=3).mean())
#train['5일_지수이동평균'] = train.groupby('종목코드')['종가'].transform(lambda x: x.ewm(span=5).mean())
#train['7일_지수이동평균'] = train.groupby('종목코드')['종가'].transform(lambda x: x.ewm(span=7).mean())
#train['10일_지수이동평균'] = train.groupby('종목코드')['종가'].transform(lambda x: x.ewm(span=10).mean())
train['15일_지수이동평균'] = train.groupby('종목코드')['종가'].transform(lambda x: x.ewm(span=15).mean())
#train['30일_지수이동평균'] = train.groupby('종목코드')['종가'].transform(lambda x: x.ewm(span=30).mean())

'''
train['종가_대비_3일_지수이동평균1'] = train.apply(lambda row: 1 if row['종가'] > row['3일_지수이동평균'] else 0, axis=1)
train['종가_대비_5일_지수이동평균1'] = train.apply(lambda row: 1 if row['종가'] > row['5일_지수이동평균'] else 0, axis=1)
train['종가_대비_7일_지수이동평균1'] = train.apply(lambda row: 1 if row['종가'] > row['7일_지수이동평균'] else 0, axis=1)
train['종가_대비_10일_지수이동평균1'] = train.apply(lambda row: 1 if row['종가'] > row['10일_지수이동평균'] else 0, axis=1)
train['종가_대비_15일_지수이동평균1'] = train.apply(lambda row: 1 if row['종가'] > row['15일_지수이동평균'] else 0, axis=1)
#train['종가_대비_30일_지수이동평균1'] = train.apply(lambda row: 1 if row['종가'] > row['30일_지수이동평균'] else 0, axis=1)
'''
'''
train['종가_대비_3일_지수이동평균'] = train['종가'] - train['3일_지수이동평균']
train['종가_대비_5일_지수이동평균'] = train['종가'] - train['5일_지수이동평균']
train['종가_대비_7일_지수이동평균'] = train['종가'] - train['7일_지수이동평균']
train['종가_대비_10일_지수이동평균'] = train['종가'] - train['10일_지수이동평균']
train['종가_대비_15일_지수이동평균'] = train['종가'] - train['15일_지수이동평균']
#train['종가_대비_30일_지수이동평균'] = train['종가'] - train['30일_지수이동평균']

'''

train['거래량_변화율'] = train.groupby('종목코드')['거래량'].pct_change()

train['3일_변동성'] = train.groupby('종목코드')['종가'].transform(lambda x: x.rolling(window=3, min_periods=1).std())
#train['5일_변동성'] = train.groupby('종목코드')['종가'].transform(lambda x: x.rolling(window=5, min_periods=1).std())
#train['7일_변동성'] = train.groupby('종목코드')['종가'].transform(lambda x: x.rolling(window=7, min_periods=1).std())
#train['10일_변동성'] = train.groupby('종목코드')['종가'].transform(lambda x: x.rolling(window=10, min_periods=1).std())
train['15일_변동성'] = train.groupby('종목코드')['종가'].transform(lambda x: x.rolling(window=15, min_periods=1).std())
#train['30일_변동성'] = train.groupby('종목코드')['종가'].transform(lambda x: x.rolling(window=30, min_periods=1).std())
from scipy.signal import convolve

def weighted_moving_average(series, period, weights=None):
    if weights is None:
        return series.rolling(period).mean()
    else:
        return series.rolling(period).apply(lambda x: np.sum(weights*x), raw=True)

def hull_moving_average(series, period):
    wma_half = weighted_moving_average(series, int(period/2))
    wma_full = weighted_moving_average(series, period)
    sqrt_period = int(np.sqrt(period))
    series = 2 * wma_half - wma_full
    return weighted_moving_average(series, sqrt_period, weights=np.arange(1, sqrt_period + 1))

# Now apply the hull moving average to the data
train['3일_HMA'] = train.groupby('종목코드')['종가'].transform(lambda x: hull_moving_average(x, 3))
#train['5일_HMA'] = train.groupby('종목코드')['종가'].transform(lambda x: hull_moving_average(x, 5))
#train['7일_HMA'] = train.groupby('종목코드')['종가'].transform(lambda x: hull_moving_average(x, 7))
#train['10일_HMA'] = train.groupby('종목코드')['종가'].transform(lambda x: hull_moving_average(x, 10))
train['15일_HMA'] = train.groupby('종목코드')['종가'].transform(lambda x: hull_moving_average(x, 15))


train


2000
1912
1912
1905


,일자,종목코드,거래량,시가,고가,저가,종가,거래량_lag_1,거래량_lag_2,거래량_lag_3,...,거래량_lag_15,3일_거래량_증가_여부,7일_거래량_증가_여부,3일_지수이동평균,15일_지수이동평균,거래량_변화율,3일_변동성,15일_변동성,3일_HMA,15일_HMA
786502,2023-01-02,A000020,189606,9400,9700,9340,9500,0.0,0.0,0.0,...,0.0,0,0,9500.000000,9500.000000,NaN,NaN,NaN,NaN,NaN
788502,2023-01-03,A000020,188464,9440,9620,9040,9200,189606.0,0.0,0.0,...,0.0,0,0,9300.000000,9340.000000,-0.006023,212.132034,212.132034,NaN,NaN
790502,2023-01-04,A000020,66861,9250,9420,9250,9330,188464.0,189606.0,0.0,...,0.0,0,0,9317.142857,9336.213018,-0.645232,150.443788,150.443788,9316.666667,NaN
792502,2023-01-05,A000020,85184,9370,9570,9350,9350,66861.0,188464.0,189606.0,...,0.0,1,0,9334.666667,9340.377581,0.274046,81.445278,122.882057,9406.666667,NaN
794502,2023-01-06,A000020,59063,9410,9440,9300,9410,85184.0,66861.0,188464.0,...,0.0,0,0,9373.548387,9358.244471,-0.306642,41.633320,110.317723,9456.666667,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978145,2023-05-23,A383800,150364,8390,8390,8310,8330,86054.0,111889.0,127668.0,...,135534.0,1,0,8344.584798,8426.375856,0.747321,15.275252,111.128411,8313.333333,49573.238095
980145,2023-05-24,A383800,122457,8310,8340,8280,8300,150364.0,86054.0,111889.0,...,67407.0,0,0,8322.292399,8410.578841,-0.185596,30.000000,115.564865,8270.000000,49525.619048
982145,2023-05-25,A383800,84241,8300,8310,8270,8310,122457.0,150364.0,86054.0,...,132215.0,0,0,8316.146200,8398.006463,-0.312077,15.275252,115.255410,8306.666667,49523.142857
984145,2023-05-26,A383800,126681,8300,8310,8270,8280,84241.0,122457.0,150364.0,...,71007.0,1,0,8298.073100,8383.255632,0.503793,15.275252,111.214079,8263.333333,49505.904762


In [83]:
train = train.drop(train.filter(regex='거래량_lag').columns, axis=1)
train

,일자,종목코드,거래량,시가,고가,저가,종가,3일_거래량_증가_여부,7일_거래량_증가_여부,3일_지수이동평균,15일_지수이동평균,거래량_변화율,3일_변동성,15일_변동성,3일_HMA,15일_HMA
786502,2023-01-02,A000020,189606,9400,9700,9340,9500,0,0,9500.000000,9500.000000,NaN,NaN,NaN,NaN,NaN
788502,2023-01-03,A000020,188464,9440,9620,9040,9200,0,0,9300.000000,9340.000000,-0.006023,212.132034,212.132034,NaN,NaN
790502,2023-01-04,A000020,66861,9250,9420,9250,9330,0,0,9317.142857,9336.213018,-0.645232,150.443788,150.443788,9316.666667,NaN
792502,2023-01-05,A000020,85184,9370,9570,9350,9350,1,0,9334.666667,9340.377581,0.274046,81.445278,122.882057,9406.666667,NaN
794502,2023-01-06,A000020,59063,9410,9440,9300,9410,0,0,9373.548387,9358.244471,-0.306642,41.633320,110.317723,9456.666667,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978145,2023-05-23,A383800,150364,8390,8390,8310,8330,1,0,8344.584798,8426.375856,0.747321,15.275252,111.128411,8313.333333,49573.238095
980145,2023-05-24,A383800,122457,8310,8340,8280,8300,0,0,8322.292399,8410.578841,-0.185596,30.000000,115.564865,8270.000000,49525.619048
982145,2023-05-25,A383800,84241,8300,8310,8270,8310,0,0,8316.146200,8398.006463,-0.312077,15.275252,115.255410,8306.666667,49523.142857
984145,2023-05-26,A383800,126681,8300,8310,8270,8280,1,0,8298.073100,8383.255632,0.503793,15.275252,111.214079,8263.333333,49505.904762


In [84]:
stock_codes = train['종목코드'].unique()

In [85]:
train

,일자,종목코드,거래량,시가,고가,저가,종가,3일_거래량_증가_여부,7일_거래량_증가_여부,3일_지수이동평균,15일_지수이동평균,거래량_변화율,3일_변동성,15일_변동성,3일_HMA,15일_HMA
786502,2023-01-02,A000020,189606,9400,9700,9340,9500,0,0,9500.000000,9500.000000,NaN,NaN,NaN,NaN,NaN
788502,2023-01-03,A000020,188464,9440,9620,9040,9200,0,0,9300.000000,9340.000000,-0.006023,212.132034,212.132034,NaN,NaN
790502,2023-01-04,A000020,66861,9250,9420,9250,9330,0,0,9317.142857,9336.213018,-0.645232,150.443788,150.443788,9316.666667,NaN
792502,2023-01-05,A000020,85184,9370,9570,9350,9350,1,0,9334.666667,9340.377581,0.274046,81.445278,122.882057,9406.666667,NaN
794502,2023-01-06,A000020,59063,9410,9440,9300,9410,0,0,9373.548387,9358.244471,-0.306642,41.633320,110.317723,9456.666667,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978145,2023-05-23,A383800,150364,8390,8390,8310,8330,1,0,8344.584798,8426.375856,0.747321,15.275252,111.128411,8313.333333,49573.238095
980145,2023-05-24,A383800,122457,8310,8340,8280,8300,0,0,8322.292399,8410.578841,-0.185596,30.000000,115.564865,8270.000000,49525.619048
982145,2023-05-25,A383800,84241,8300,8310,8270,8310,0,0,8316.146200,8398.006463,-0.312077,15.275252,115.255410,8306.666667,49523.142857
984145,2023-05-26,A383800,126681,8300,8310,8270,8280,1,0,8298.073100,8383.255632,0.503793,15.275252,111.214079,8263.333333,49505.904762


In [86]:
def create_model():
    model = Sequential()
    model.add(Conv1D(64, kernel_size=15, activation='relu', input_shape=(30, 11)))
    model.add(Conv1D(32, kernel_size=15, activation='relu'))
    model.add(LSTM(32, activation='tanh', return_sequences=False))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(15))
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])
    return model

In [87]:
train = train.dropna()
train

,일자,종목코드,거래량,시가,고가,저가,종가,3일_거래량_증가_여부,7일_거래량_증가_여부,3일_지수이동평균,15일_지수이동평균,거래량_변화율,3일_변동성,15일_변동성,3일_HMA,15일_HMA
818502,2023-01-26,A000020,52294,9260,9280,9220,9270,0,0,9237.617017,9278.228737,-0.242610,37.859389,104.980724,9313.333333,54922.095238
820502,2023-01-27,A000020,55507,9290,9330,9240,9290,0,0,9263.808608,9279.846371,0.061441,41.633320,105.143892,9323.333333,54808.666667
822502,2023-01-30,A000020,65513,9300,9370,9300,9330,0,0,9296.904367,9286.654031,0.180266,30.550505,104.790040,9363.333333,54853.333333
824502,2023-01-31,A000020,54961,9350,9400,9280,9380,0,0,9338.452223,9299.189867,-0.161067,45.092498,103.085725,9426.666667,55087.523810
826502,2023-02-01,A000020,64283,9410,9450,9370,9430,0,0,9384.226133,9316.595157,0.169611,50.000000,96.619036,9480.000000,55476.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978145,2023-05-23,A383800,150364,8390,8390,8310,8330,1,0,8344.584798,8426.375856,0.747321,15.275252,111.128411,8313.333333,49573.238095
980145,2023-05-24,A383800,122457,8310,8340,8280,8300,0,0,8322.292399,8410.578841,-0.185596,30.000000,115.564865,8270.000000,49525.619048
982145,2023-05-25,A383800,84241,8300,8310,8270,8310,0,0,8316.146200,8398.006463,-0.312077,15.275252,115.255410,8306.666667,49523.142857
984145,2023-05-26,A383800,126681,8300,8310,8270,8280,1,0,8298.073100,8383.255632,0.503793,15.275252,111.214079,8263.333333,49505.904762


In [88]:
train.drop(['시가','고가','저가'],axis=1,inplace=True)
train

C:\Users\ineeji\AppData\Local\Temp\ipykernel_196980\3936775641.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(['시가','고가','저가'],axis=1,inplace=True)


,일자,종목코드,거래량,종가,3일_거래량_증가_여부,7일_거래량_증가_여부,3일_지수이동평균,15일_지수이동평균,거래량_변화율,3일_변동성,15일_변동성,3일_HMA,15일_HMA
818502,2023-01-26,A000020,52294,9270,0,0,9237.617017,9278.228737,-0.242610,37.859389,104.980724,9313.333333,54922.095238
820502,2023-01-27,A000020,55507,9290,0,0,9263.808608,9279.846371,0.061441,41.633320,105.143892,9323.333333,54808.666667
822502,2023-01-30,A000020,65513,9330,0,0,9296.904367,9286.654031,0.180266,30.550505,104.790040,9363.333333,54853.333333
824502,2023-01-31,A000020,54961,9380,0,0,9338.452223,9299.189867,-0.161067,45.092498,103.085725,9426.666667,55087.523810
826502,2023-02-01,A000020,64283,9430,0,0,9384.226133,9316.595157,0.169611,50.000000,96.619036,9480.000000,55476.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...
978145,2023-05-23,A383800,150364,8330,1,0,8344.584798,8426.375856,0.747321,15.275252,111.128411,8313.333333,49573.238095
980145,2023-05-24,A383800,122457,8300,0,0,8322.292399,8410.578841,-0.185596,30.000000,115.564865,8270.000000,49525.619048
982145,2023-05-25,A383800,84241,8310,0,0,8316.146200,8398.006463,-0.312077,15.275252,115.255410,8306.666667,49523.142857
984145,2023-05-26,A383800,126681,8280,1,0,8298.073100,8383.255632,0.503793,15.275252,111.214079,8263.333333,49505.904762


In [89]:
results = []
for stock_code in tqdm(stock_codes, desc="Training Progress"):
    stock_data = train[train['종목코드'] == stock_code]
    input_data = stock_data.iloc[:,2:].values
    window_size = 30
    forecast_days = 15

    x = []
    y = []
    last_window = input_data[-window_size:].reshape(1, window_size, 11)

    for i in range(window_size, len(input_data)-forecast_days):
        x.append(input_data[i-window_size:i, :])
        y.append(input_data[i:i+forecast_days, 1])

    x = np.array(x)
    y = np.array(y)
    print(x.shape)
    # 1-3. scaler (2차원만 가능하기에 reshape)
    x = np.reshape(x, (x.shape[0], x.shape[1]*x.shape[2]))
    last_window = np.reshape(last_window, (last_window.shape[0], last_window.shape[1]*last_window.shape[2]))


    #scaler = MinMaxScaler()
    # scaler = StandardScaler()
    #scaler.fit(x)
    #x = scaler.transform(x)
    #last_window = scaler.transform(last_window)

    # 1-4. reshape
    x = np.reshape(x, (x.shape[0], window_size, 11))
    last_window = np.reshape(last_window, (last_window.shape[0], window_size, 11))

    model = create_model()
    #from keras.callbacks import EarlyStopping
    #earlyStopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
    #result = model.fit(x, y, epochs=1000, batch_size=99999, verbose=0, callbacks=[earlyStopping])
    result = model.fit(x, y, epochs=1000, batch_size=999999, verbose=0)

    submit = model.predict(last_window)

    ratio = [0] * 14
    '''
    for i in range(14):
        ratio[i] = (submit[0][i+1]-submit[0][i])/submit[0][i] * 100

    sum_ratio = sum(ratio)
    '''
    sum_ratio = submit[0][14]/submit[0][0]
    results.append({
        '종목코드' : stock_code,
        'sum_ratio': sum_ratio,
    })

Training Progress:   0%|          | 0/1905 [00:00<?, ?it/s]

(40, 30, 11)
1/1 [==============================] - 0s 193ms/step


Training Progress:   0%|          | 1/1905 [00:23<12:24:59, 23.48s/it]

(40, 30, 11)
1/1 [==============================] - 0s 219ms/step


Training Progress:   0%|          | 2/1905 [00:46<12:09:27, 23.00s/it]

(40, 30, 11)


Training Progress:   0%|          | 2/1905 [01:04<17:00:22, 32.17s/it]


KeyboardInterrupt: 

In [ ]:
for result in results:
    print('종목코드', result['종목코드'])
    print('sum_ratio', result['sum_ratio'])

results_df = pd.DataFrame(results)
results_df.to_csv('./Conv1d_LSTM_30_spiltx.csv', index=False)

종목코드 A000020
sum_ratio 0.97366923
종목코드 A000040
sum_ratio 0.99289227
종목코드 A000050
sum_ratio 1.210552
종목코드 A000070
sum_ratio 1.0652766
종목코드 A000080
sum_ratio 0.9413233
종목코드 A000100
sum_ratio 1.0600407
종목코드 A000120
sum_ratio 1.0097231
종목코드 A000140
sum_ratio 0.8519665
종목코드 A000150
sum_ratio 0.9014645
종목코드 A000180
sum_ratio 1.0252544
종목코드 A000210
sum_ratio 1.018261
종목코드 A000220
sum_ratio 0.90678674
종목코드 A000230
sum_ratio 1.0534141
종목코드 A000240
sum_ratio 1.1285787
종목코드 A000250
sum_ratio 0.9740215
종목코드 A000270
sum_ratio 1.0828706
종목코드 A000300
sum_ratio 0.9850625
종목코드 A000320
sum_ratio 0.9154694
종목코드 A000370
sum_ratio 0.96439135
종목코드 A000390
sum_ratio 1.0473481
종목코드 A000400
sum_ratio 0.9864897
종목코드 A000430
sum_ratio 0.8373179
종목코드 A000440
sum_ratio 0.8667197
종목코드 A000490
sum_ratio 1.0296483
종목코드 A000500
sum_ratio 0.8691162
종목코드 A000520
sum_ratio 0.91397196
종목코드 A000540
sum_ratio 0.95844114
종목코드 A000590
sum_ratio 0.9352992
종목코드 A000640
sum_ratio 1.1257616
종목코드 A000650
sum_ratio 0.96216553
종목코드 

In [ ]:
results_df.describe()

,sum_ratio
count,1812.000000
mean,1.000079
std,0.093519
min,0.677160
25%,0.937288
50%,0.994691
75%,1.055657
max,1.409601


In [ ]:
d = dropped_samples.copy()
unique_series = d['종목코드'].drop_duplicates()
a = results_df.copy()
a['순위'] = a['sum_ratio'].rank(method='first', ascending=False).astype('int')
unique_series

295481    A000480
294272    A001140
294463    A001230
294430    A001440
294032    A001530
           ...   
295769    A319660
294998    A332370
295775    A347740
295690    A356860
294035    A375500
Name: 종목코드, Length: 188, dtype: object

In [ ]:
# 'unique_series'를 데이터프레임으로 변환
df_unique_series = unique_series.to_frame()

# 'a'와 'df_unique_series'를 결합
df = pd.concat([a, df_unique_series])

# 'a'의 원래 '순위' 값들 중 1000 이상인 경우에만 'df_unique_series'의 길이만큼 순위를 증가시킵니다.
df.loc[(df.index.isin(a.index)) & (df['순위'] >= 1000), '순위'] += len(df_unique_series)

# 'df_unique_series'에는 1000부터 시작하는 순위를 부여합니다.
df.loc[df_unique_series.index, '순위'] = range(1000, 1000 + len(df_unique_series))

# '순위'를 정수로 변환
df['순위'] = df['순위'].astype(int)
df

,종목코드,sum_ratio,순위
0,A000020,0.973669,1277
1,A000040,0.992892,925
2,A000050,1.210552,44
3,A000070,1.065277,395
4,A000080,0.941323,1517
...,...,...,...
295769,A319660,NaN,1183
294998,A332370,NaN,1184
295775,A347740,NaN,1185
295690,A356860,NaN,1186


In [ ]:
a[a['순위'] == 1811]

,종목코드,sum_ratio,순위
220,A005680,0.722532,1811


In [ ]:
df[df['순위'] == 1999]

,종목코드,sum_ratio,순위
220,A005680,0.722532,1999


In [ ]:
df.drop(['sum_ratio'],axis=1,inplace=True)
df.to_csv('lstm_conv1d_2023.csv',index=False)
df

,종목코드,순위
0,A000020,1277
1,A000040,925
2,A000050,44
3,A000070,395
4,A000080,1517
...,...,...
295769,A319660,1183
294998,A332370,1184
295775,A347740,1185
295690,A356860,1186
